# Look at WG met file (subset of WG L2 data file)



## Working toward estimation of bulk fluxes

Will need:  
- lat/lon (1 Hz and 20 Hz)
- SMP21/SGR4 data
- wind (u,v,w)
- all WXT variables
- SST, SSS, sensor depth ("UCTD")

1 Hz variables I need:
- time_1Hz
- latitude_1Hz
- longitude_1Hz
- UCTD_sea_water_temperature
- UCTD_salinity
- UCTD_density
- UCTD_depth
- WXT_rainfall_rate
- WXT_atmospheric_pressure
- WXT_air_temperature
- WXT_relative_humidity
- WXT_wind_speed
- WXT_wind_direction
- SMP21_shortwave_flux
- SGR4_longwave_flux
- _also need some estimate of surface current_

20 Hz variables I need:
- time_20Hz
- latitude_20Hz
- longitude_20Hz
- wind_speed
- wind_direction
- wind_vertical

Not sure if I need:  
- time_15min
- wave_frequency


In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib
import datetime as dt
################
# This allows us to import Tom_tools
import sys
sys.path.append('../../Tom_tools/') # you may need to adjust this path
# sys.path.append('../SWOT_IW_SSH/jtf/Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################

In [2]:
# %matplotlib inline
%matplotlib qt 
savefig = True # set to true to save plots as file
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 400
plt.close('all')
plt.rcParams['axes.xmargin'] = 0


__figdir__ = '../plots/WG_timeseries/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
path = '/mnt/e/SMODE_data/pilot/WG/L2/'
WG = 'WHOI43'
file = 'SMODE_PFC_Wavegliders_'+WG+'.nc'

In [4]:
# This needs to be set before any dates are encoded as datetime64
# Because the time increment is so small, we get 'out of range' when starting from the standard datetime64 epoch of 1970
# matplotlib.dates.set_epoch('2000-01-01T00:00:00') 

In [5]:
%%time
ds = xr.open_dataset(path+file, engine = 'netcdf4', decode_times = True) #decode_times = False, 

CPU times: user 428 ms, sys: 153 ms, total: 581 ms
Wall time: 1.61 s


In [6]:
ds

<xarray.Dataset>
Dimensions:                     (time_1Hz: 771961, time_10Hz: 7719601, time_15min: 858, wave_frequency: 153, time_20Hz: 15439201, Workhorse_time: 771960, Workhorse_range: 21)
Coordinates: (12/16)
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T00:59:59...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_10Hz                   (time_10Hz) datetime64[ns] 2021-10-28T00:59:5...
    altitude_10Hz               (time_10Hz) float32 ...
  * time_15min                  (time_15min) datetime64[ns] 2021-10-28T00:59:...
    ...                          ...
    altitude_20Hz               (time_20Hz) float32 ...
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T01...
  * Workhorse_range             (Workhorse_range) float32 3.93 5.93 ... 43.93
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 ...
Data variables: (12/47)
    INST_RBR_Concerto           float64 ...
    INST_Gill                   float64 ...
    INST_SMP21                  float64 ...
    INST_SGR4                   float64 ...
    INST_upper_ctd              float64 ...
    INST_WXT                    float64 ...
    ...                          ...
    Workhorse_roll              (Workhorse_time) float32 ...
    Workhorse_temperature       (Workhorse_time) float32 ...
    Workhorse_vel_east          (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_north         (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_vel_up            (Workhorse_time, Workhorse_range) float32 ...
    Workhorse_flag              (Workhorse_time, Workhorse_range) float32 ...
Attributes: (12/40)
    DOI:                           10.5067/SMODE-GLID3
    acknowledgement:               This work is a contribution to the S-MODE ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    summary:                       Wave Glider WHOI43 was deployed by R/V Oce...
    uuid:                          41d0900b-f3ec-4013-9c1d-b758a6b77ba4
    history:                       2022-04-11 Benjamin Greenwood - process in...
    date_created:                  2022-04-11T19:46:12Z
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    instruments:                   In Situ/Laboratory Instruments > Recorders...

In [7]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Raw 1 Hz WXT measurements')

plt.subplot(5,1,2)
plt.plot(ds.time_1Hz, ds.WXT_relative_humidity)
plt.ylabel('[%]')
plt.legend(['Rel. Humidity'])

plt.subplot(5,1,3)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

plt.subplot(5,1,4)
plt.plot(ds.time_20Hz, ds.wind_speed)
plt.plot(ds.time_1Hz, ds.WXT_wind_speed)
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.Workhorse_time, ds.Workhorse_altitude)
plt.ylabel('[m]')
plt.legend(['Workhorse altitude (from IMU/GPS)'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met' + '.' +plotfiletype,**savefig_args)


In [8]:
# Raw met plot from WG:
fig, axs = plt.subplots(4, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(4,1,1)
plt.plot(ds.time_1Hz, ds.WXT_atmospheric_pressure)
plt.ylabel('[mbar]')
plt.legend(['Atm. pressure'])

plt.subplot(4,1,2)
plt.plot(ds.time_1Hz, ds.SGR4_longwave_flux)
plt.ylabel('W/m^2')
plt.legend(['Longwave radiation'])

plt.subplot(4,1,3)
plt.plot(ds.time_1Hz, ds.SMP21_shortwave_flux)
plt.ylabel('W/m^2')
plt.legend(['Shortwave radiation'])

plt.subplot(4,1,4)
plt.plot(ds.time_1Hz, ds.WXT_rainfall_rate)
plt.ylabel('[mm/hr]')
plt.legend(['Precip. rate'])

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met2' + '.' +plotfiletype,**savefig_args)


In [18]:
# Raw met plot from WG:
fig, axs = plt.subplots(5, 1, sharex=True,figsize=(5,7))
fig.autofmt_xdate()
plt.subplot(5,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Wave Glider surface measurements')

plt.subplot(5,1,2)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SGR4_longwave_flux,15*60))
plt.ylabel('W/m^2')
plt.legend(['Longwave radiation'])
plt.setp(plt.gca().get_xticklabels(), visible=False)

plt.subplot(5,1,3)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.SMP21_shortwave_flux,15*60))
plt.ylabel('W/m^2')
plt.legend(['Shortwave radiation'])

plt.subplot(5,1,4)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['wind speed'],loc='upper right')

plt.subplot(5,1,5)
plt.plot(ds.time_15min, ds.wave_significant_height)
plt.ylabel('[m]')
plt.legend(['Sig. wave height'],loc='upper right')

if savefig:
    plt.savefig(__figdir__+WG+'_raw_met3' + '.' +plotfiletype,**savefig_args)


In [ ]:
# There is a limited number of NaNs in WXT wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz, np.isnan(ds.WXT_wind_speed.values))
plt.title('NaNs in WXT wind speed')

In [ ]:
# Let's interpolate them
ff = np.where(np.isnan(ds.WXT_wind_speed.values)==0)
t = ds.time_1Hz[ff]
wspd = ds.WXT_wind_speed[ff]
WXT_wspd_interp = np.interp(ds.time_1Hz, t, wspd)

In [ ]:
'''
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

# plt.plot(t, np.isnan(wspd))
plt.plot(np.isnan(WXT_wspd_interp))
plt.title('NaNs in WXT wind speed after trying to remove nans')
''' # this checks out

In [ ]:
# There is also a limited number of NaNs in Gill wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_20Hz, np.isnan(ds.wind_speed.values))
plt.title('NaNs in Gill wind speed')

In [ ]:
# Let's interpolate them
ff = np.where(np.isnan(ds.wind_speed.values)==0)
t = ds.time_20Hz[ff]
wspd = ds.wind_speed[ff]
wspd_interp = np.interp(ds.time_20Hz, t, wspd)

In [ ]:
fig, axs = plt.subplots(1, 1)
M=151
tt.spectrum_band_avg(wspd_interp,1/20,M,winstr=None,plotflag=True,ebarflag=None)
tt.spectrum_band_avg(WXT_wspd_interp,1,M,winstr=None,plotflag=True,ebarflag=False)
plt.title('Gill, WXT wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')
if savefig:
    plt.savefig(__figdir__+WG+'_wsp_spectra' + '.' +plotfiletype,**savefig_args)


## There appears to be some residual wave contamination in the motion corected velocities, but it shouldn't be a big issue when we average to 1 min or longer

In [ ]:
fig, axs = plt.subplots(1, 1, sharex=True)
fig.autofmt_xdate()
plt.plot(ds.time_20Hz, tt.run_avg1d(ds.wind_speed,15*60*20))
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed ('+str(ds.INST_Gill.height_m)+'m)','WXT wind speed ('+str(ds.INST_WXT.height_above_waterline_m)+'m)'],loc='upper right')
plt.title('15-min average Gill R-3 and WXT wind speed')
plt.axis('tight')
if savefig:
    plt.savefig(__figdir__+WG+'_wspd_smoothed' + '.' +plotfiletype,**savefig_args)


In [ ]:
str(ds.INST_Gill.height_m)

In [ ]:
ds.INST_WXT.height_above_waterline_m

In [ ]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.Workhorse_time, ds.Workhorse_altitude)
plt.title('Workhorse altitude\n Something bad happened with the VectorNav?')
plt.xlim(18935.65223174665, 18936.366246499798)

if savefig:
    plt.savefig(__figdir__+WG+'_Workhorse_altitude' + '.' +plotfiletype,**savefig_args)


In [ ]:
xlim = plt.xlim()

In [ ]:
xlim

## Make an averaged data set (say, 1 min)

It would probably be smarter to automate this to loojp through all ds.time_1Hz variables (for example).  That might look something like this:  
```
var_list = []  
for var in ds.data_vars.variables:  
    var_list.append(var)  
    print(var)  
```

In [ ]:
# compare different time vectors to decide how to line up variables from different time bases
# ds.time_20Hz
# ds.time_1Hz
# ds.Workhorse_time

# Try the simple thing
# xy, x_ind, y_ind = np.intersect1d(ds.time_1Hz, ds.time_20Hz, assume_unique=True, return_indices=True)
# That doesn't work (no surprise) because the times don't line up exactly (possibly numerical precision issue)

# I think they are already nearly lined up
ds.time_1Hz[0:10:1]-ds.time_20Hz[0:200:20]
# so, try 0,20,40... in 20 Hz array
tfoo=[]
for n in range(len(ds.time_1Hz)):
    tfoo.append(ds.time_20Hz[0:200:20]

In [ ]:
len(ds.time_1Hz)

In [ ]:
nsec=60*15 # 15 minute average
# ds.time_20Hz variables
# wdir is positive clockwise from North
# need to make U, V
Gill_U=ds.wind_speed*np.cos(ds.wind_direction/np.pi/2)
Gill_V=ds.wind_speed*np.sin(ds.wind_direction/np.pi/2)
Gill_wspd_low = tt.run_avg1d(ds.wind_speed, nsec*20)
Gill_U_low = tt.run_avg1d(Gill_U,nsec*20)
Gill_V_low = tt.run_avg1d(Gill_V,nsec*20)
pitch_low = tt.run_avg1d(ds.pitch,nsec*20)
roll_low = tt.run_avg1d(ds.roll,nsec*20)

# ds.time_1Hz variables
WXT_U=ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction/np.pi/2)
WXT_V=ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction/np.pi/2)
WXT_wspd_low = tt.run_avg1d(ds.WXT_wind_speed,nsec)
WXT_atmp_low = tt.run_avg1d(ds.WXT_air_temperature,nsec)
WXT_rh_low = tt.run_avg1d(ds.WXT_relative_humidity,nsec)
swr_low = tt.run_avg1d(ds.SMP21_shortwave_flux,nsec)
lwr_low = tt.run_avg1d(ds.SGR4_longwave_flux,nsec)
lat_low = tt.run_avg1d(ds.latitude_1Hz,nsec)
lon_low = tt.run_avg1d(ds.longitude_1Hz,nsec)

# Workhorse_time variables
U_low=tt.run_avg2d(ds.Workhorse_vel_east,nsec,1)
V_low=tt.run_avg2d(ds.Workhorse_vel_north,nsec,1)
W_low=tt.run_avg2d(ds.Workhorse_vel_up,nsec,1)

## Preparing for flux computation

Inputs for COARE 3.5:  
    u = ocean surface relative wind speed (m/s) at height zu(m)  
    t = bulk air temperature (degC) at height zt(m)  
    rh = relative humidity (%) at height zq(m)  
    ts = sea water temperature (degC) - see jcool below  
    P = surface air pressure (mb) (default = 1015)  
    Rs = downward shortwave radiation (W/m^2) (default = 150)  
    Rl = downward longwave radiation (W/m^2) (default = 370)  
    zu = wind sensor height (m) (default = 18m)  
    zt = bulk temperature sensor height (m) (default = 18m)  
    zq = RH sensor height (m) (default = 18m)  
    lat = latitude (default = 45 N)  
    zi = PBL height (m) (default = 600m)  
    rain = rain rate (mm/hr)  
    cp = phase speed of dominant waves (m/s)  
    sigH =  significant wave height (m)  
    jcool = cool skin option (default = 1 for bulk SST)  
    
Note: I don't see an input for SST depth--> That's because there is no Warm Layer correction in this version of the code.  

Here is a link to an older matlab version that does have it:  
https://github.com/carsonwitte/Falkor-DWL-Code  

It wouldn't be that hard to add the WL correction-- basically, estimate fluxes, estimate WL correction, and re-estimate fluxes.  


  
  
Required inputs:  
u	zu	t	zt	rh	zq	P	ts	Rs	Rl	lat	zi	rain	cp	sigH


This could be a good way to save the output, but maybe not:  

```
A = coare35vn(u, ta, rh, ts, P=Pa, Rs=rs, Rl=rl, zu=16, zt=16, zq=16,
                lat=Lat, zi=ZI, rain=Rain, jcool=1)
fnameA = os.path.join(path,'test_35_output_py_04022022.txt')
A_hdr = 'usr\ttau\thsb\thlb\thlwebb\ttsr\tqsr\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzet\tdter\tdqer\ttkt\tRF\tCdn_10\tChn_10\tCen_10'
np.savetxt(fnameA,A,fmt='%.18e',delimiter='\t',header=A_hdr)
```
